## Aufgabe 2:

Gibt es „Angstgegner“ einzelner deutscher Spieler, d.h. Paarungen, bei denen der deutsche
Spieler schlechter abschneidet, als es aufgrund der allgemeinen Spielstärke beider Spieler zu
erwarten ist? Gibt es analog „Lieblingsgegner“ einzelner deutscher Spieler?

##### Import der genutzten Python-Bibliotheken

In [1]:
import numpy as np
import pandas as pd

##### Import der benötigten Daten und Erstellung einer Kopie mit der Teilmenge an Matches an denen min. ein deutscher Spieler beteiligt gewesen ist

In [2]:
df = pd.read_csv('/home/ronja/documents/studium/bi-project/data/atp_matches_till_2022_preprocessed.csv', delimiter=';', low_memory=False)
df_2 = df.loc[(df['loser_ioc'] == 'GER') | (df['winner_ioc'] == 'GER')].copy()

##### Ermitteln der Matches bei denen die Kontrahenten ein ähnliches Skill-Level haben
Als Anhaltspunkt für das Skill-Level wird der beim Matchtag aktuelle Stand der Rangpunkte von der ATP-Weltrangliste genommen. Um den Umfang des Skill-Unterschiedes greifbar zu machen wird der Betrag der Differenz zwischen den Rangpunkten beider Kontrahenten berechnet und darauf geprüft, ob dieser kleiner oder gleich dem als Grenzwert gewählten Median ist. Wenn das der Fall ist, wird dieses Aufeinandertreffen in eine neue Tabelle zur weiteren Betrachtung übernommen.

In [3]:
# Calculate the absolute difference between the two rank points columns
df_2['diff'] = np.abs(df_2['winner_rank_points'] - df_2['loser_rank_points'])
df_2['diff'] = df_2['diff'].dropna()

# Calculate the allowed rank points difference between the two opponents (0.5 ==> median) 
quantile = df_2['diff'].quantile(q=0.5)

# Select the rows where the difference is less than or equal to the quantile var
df_3 = df_2[df_2['diff'] <= quantile]

##### Erstellung einer Pivot Tabelle, zur Darstellung der unterschiedlichen Win/Lose-Verhältnisse zwischen der Kontrahentenpaaren
Zuerst werden zwei Dataframes erstellt, die die Informationen über den Ausgang der Begegnungen enthalten. Dies werden zusammengefügt, gruppiert und darauf überprüft, ob eine Mindestanzahl an Begegnungen stattgefunden hat.

In [19]:
# Minimal amount of matches between two opponents
min_matches = 6

# Create a dataframe which includes all matches won by Player1
wins = pd.DataFrame({'Player1': df_3['winner_name'], 'Player2': df_3['loser_name'], 'Result': 1})

# Create a dataframe which includes all matches won by Player2
losses = pd.DataFrame({'Player1': df_3['loser_name'], 'Player2': df_3['winner_name'], 'Result': 0})

# Merge the wins and losses dfs
all_matches = pd.concat([wins, losses])

# Group by 'Player1' and 'Player2' and remove the pairs that played less than min_matches against each other
all_matches = all_matches.groupby(['Player1', 'Player2']).filter(lambda group: len(group) >= min_matches)

# Create a pivot table with the win/loss-ratio for every pair in the all matches table
pivot_table = pd.pivot_table(all_matches, values='Result', index=['Player1'], columns=['Player2'])

##### Ausgabe der Pivot Tabelle
Ein Wert von 1 bedeutet, dass der Zeilenspieler(Player1) immer gewonnen hat und ein Wert von 0,5 bedeutet, dass sie die Hälfte der Matches gewonnen haben. 

In [28]:
def make_pretty(styler):
    styler.background_gradient(axis=None, vmin=0, vmax=1, cmap="YlGnBu")
    return styler


pivot_table.style.pipe(make_pretty)

Player2,Andreas Seppi,Andrei Pavel,Arnaud Boetsch,Benjamin Becker,Carl Uwe Steeb,Carlos Moya,Denis Istomin,Dominik Hrbaty,Fabrice Santoro,Florian Mayer,Gilles Simon,Igor Andreev,Ivan Ljubicic,Ivo Karlovic,Jarkko Nieminen,Jim Courier,Juan Monaco,Jurgen Melzer,Michael Stich,Nicolas Kiefer,Olivier Rochus,Philipp Kohlschreiber,Rainer Schuettler,Tommy Haas,Vincent Spadea
Player1,,,,,,,,,,,,,,,,,,,,,,,,,
Andreas Seppi,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.375000,nan,nan,nan
Andrei Pavel,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.333333,nan
Arnaud Boetsch,nan,nan,nan,nan,0.428571,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Benjamin Becker,nan,nan,nan,nan,nan,nan,0.142857,nan,nan,nan,nan,nan,nan,0.125000,0.250000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Carl Uwe Steeb,nan,nan,0.571429,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Carlos Moya,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.333333,nan,nan
Denis Istomin,nan,nan,nan,0.857143,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dominik Hrbaty,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.500000,nan
Fabrice Santoro,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.333333,nan
